# Projet Tri de déchets - Classification de déchets sur photos

Ce notebook permet de réaliser une classification de déchets sur une photo à partir de modèles de CNN entrainés et sauvegardés sur un *Google Drive* connecté.

### Membres de l'équipe :
- Amine CHERIF HAOUAT
- Tom LABIAUSSE
- Cyrine NABI
- Pierre OLLIVIER
- Selim BEN TURKIA

# 0 - Installation des packages

Les cellules suivantes doivent être exécutées uniquement si l'utilisateur rencontre des difficultés lors de l'importation des modules en section 1.


In [ ]:
# KERAS
!pip install keras==2.4.3

In [ ]:
# TENSORFLOW
!pip install tensorflow==2.4.1

In [ ]:
# PYDRIVE
!pip install PyDrive

# 1 - Les imports

Exécutez les trois cellules suivantes dans l'ordre en suivant les instructions :

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print("-> Authentification Google OK")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
path_to_GGDrive = "/content/gdrive/MyDrive/"
print("-> Connexion à Google Drive OK")

In [ ]:
from os import listdir
import tensorflow as tf
import numpy as np

from tensorflow import keras

from tensorflow.keras import datasets, layers, models, losses, optimizers
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
# import matplotlib
# import sys

from PIL import Image
from PIL import ImageFile
from PIL import ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten, Dropout, BatchNormalization

from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
print("-> Importations OK.")

# 2 - Fonctionnalités diverses

Exécutez les trois cellules suivantes dans l'ordre

In [4]:
def to_percent(x,nb_dec=0):
    """ Donne x sous la forme de pourcentage avec 'nb_dec' chiffres apres la virgule. """
    if nb_dec == 0:
        return(int(100*x))
    else:
      return( float(int((10**(2+nb_dec))*x))/(10**nb_dec) )

In [5]:
def already_exists(file_name,place="."):
    """ Renvoie True si 'file_name' existe à l'emplacement 'place' et False sinon. """
    existing_files = listdir(place)
    for a_file in existing_files:
        if a_file == file_name:
            return(True)
    return(False)

In [6]:
def get_dico_pred(nom_image,dico_cnn):
    """ Renvoie un dictionnaire dont les clefs sont les catégories de déchets et les valeurs les prédictions associées. """
    ## Preprocessing de l'image
    img = load_img(path_to_GGDrive + nom_image, target_size=(224, 224))  # Charger l'image
    img = img_to_array(img)  # Convertir en tableau numpy de shape (224,224,3)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))  # Créer la collection d'images (un seul échantillon)
    img = preprocess_input(img)  # Prétraiter l'image comme le veut VGG-16
    ## Réalise les prédictions pour chaque classifieur binaire
    dico_pred = {}
    for (name,model_cnn) in dico_cnn.items():
        dico_pred[name] = to_percent(model_cnn.predict(img)[0][0],nb_dec=2)
    return(dico_pred)

# 3 - Charger les modèles de CNN

Entrez entre guillemets le nom du dossier contenant les modèles de CNN :

In [7]:
dossier_modeles = "SAVED_MODELS"
dossier_modeles += "/"

Entrez entre guillemets le nom des fichiers modèles de CNN :

(Si un modèle n'est pas disponible, écrivez seulement deux guillemets consécutifs : "")

In [8]:
dico_models = {}
dico_models["bottle"] = "CNN_bottle.h5"
dico_models["can"] = "CNN_can.h5"
dico_models["cup"] = "CNN_cup.h5"
dico_models["carton"] = "CNN_carton.h5"
dico_models["cigarette"] = "CNN_cigarette.h5"
dico_models["mask"] = "CNN_mask.h5"
dico_models["paper"] = "CNN_paper.h5"
dico_models["plastic_bag"] = "CNN_plastic_bag.h5"
dico_models["other_plastic"] = "CNN_other_plastic.h5"
dico_models["styrofoam"] = "CNN_styrofoam.h5"

Exécutez la cellule ci-dessous pour charger les modèles spécifiés :

In [ ]:
dico_cnn = {}
for (name,model_name) in dico_models.items():
    if model_name != "":
        if not(already_exists(model_name,place=path_to_GGDrive + dossier_modeles)):
            print("""Modele "{0}" introuvable à l'emplacement '{1}'.""".format(model_name,path_to_GGDrive + dossier_modeles))
        else:
            dico_cnn[name] = tf.keras.models.load_model(path_to_GGDrive + dossier_modeles + model_name)
            print("""Modele "{0}" chargé.""".format(model_name))

# 4 - Réaliser une classification sur une image

Entrez entre guillemets le nom de l'image :

In [11]:
nom_image = "je_suis_une_image.jpg"

Visualisez l'image :

In [ ]:
pil_image =Image.open(path_to_GGDrive + nom_image)
plt.imshow(pil_image) ; plt.title(nom_image) ; plt.show()
print(" ")
print("Largeur : " + str(pil_image.width) + " | Hauteur : " + str(pil_image.height))
print("Mode : " + pil_image.mode + " | Format : " + pil_image.format)

Réalisez la classification :

In [ ]:
seuil_1 = 0.5 ; seuil_2 = 0.65
dico_pred = get_dico_pred(nom_image,dico_cnn)
print("\nPrédictions sur '" + nom_image + "' :")
for (name,pred) in dico_pred.items():
    if pred < seuil_1:
        region = "NON"
    elif seuil_1 <= pred < seuil_2:
        region = "INDECIS"
    else:
       region = "OUI"
    print("- " + name.upper() + " : " + region + " (" + str(pred) + "%)")